In [ ]:
# Install pyfuzzy-toolbox with ML module and scikit-learn
!pip install pyfuzzy-toolbox[ml] scikit-learn -q

In [ ]:
import fuzzy_systems as fs
from fuzzy_systems.learning import ANFIS
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['figure.dpi'] = 100

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import pandas as pd

In [ ]:
# Load dataset
iris = load_iris()

X = iris.data  # Shape (150, 4) - 4 features
y = iris.target
y_binary = (y != 0).astype(int)  # 0 = Setosa, 1 = Non-Setosa

feature_names = iris.feature_names 
class_names = iris.target_names

# Create DataFrame
df = pd.DataFrame(X, columns=iris.feature_names)
df['species'] = iris.target_names[y]

print('Iris dataset loaded!')
print(f'\nShape: {X.shape}')
print(f'\nClasses: {iris.target_names}')
print(f'\nFeatures: {iris.feature_names}')
df.head()

In [ ]:
# Visualize feature distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, feature in enumerate(iris.feature_names):
    for species_id, species_name in enumerate(iris.target_names):
        data = df[df['species'] == species_name][feature]
        axes[idx].hist(data, alpha=0.6, bins=15, label=species_name)

    axes[idx].set_xlabel(feature, fontsize=11)
    axes[idx].set_ylabel('Frequency', fontsize=11)
    axes[idx].set_title(f'Distribution: {feature}', fontsize=12, weight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Prepare Data - Binary Classification (Setosa vs Non-Setosa)
X = X[:, 2:4]
feature_names = ['Petal Length (cm)', 'Petal Width (cm)']
class_names = ['setosa', 'versicolor', 'virginica']

# Visualize separability
fig, ax = plt.subplots(figsize=(10, 8))

colors = ['red', 'blue']
labels = ['Setosa', 'Non-Setosa']

for class_id in [0, 1]:
    mask = y_binary == class_id
    ax.scatter(X[mask, -2], X[mask, -1], 
              c=colors[class_id], label=labels[class_id],
              s=80, alpha=0.7, edgecolors='black', linewidth=1)

ax.set_xlabel(iris.feature_names[-2] + ' (cm)', fontsize=13)
ax.set_ylabel(iris.feature_names[-1] + ' (cm)', fontsize=13)
ax.set_title('Iris Dataset - 2 Features (Petal)', fontsize=15, weight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('Note: Setosa is clearly separable from other classes!')

In [ ]:
# Binary Classification Example
print("=" * 70)
print("ANFIS BINARY CLASSIFICATION")
print("=" * 70)

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_binary, test_size=0.3, random_state=42, stratify=y_binary
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Normalize data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

for i in range(X_train.shape[1]):
    print(f'Feature {i+1} - Train: [{X_train_norm[:, i].min():.2f}, {X_train_norm[:, i].max():.2f}] | '
          f'Validation: [{X_val_norm[:, i].min():.2f}, {X_val_norm[:, i].max():.2f}] | '
          f'Test: [{X_test_norm[:, i].min():.2f}, {X_test_norm[:, i].max():.2f}]')

In [ ]:
# Create and train ANFIS model for classification
num_inputs = len(X_train_norm[0, :])
input_ranges = [[X_train_norm[:, idx].min(), X_train_norm[:, idx].max()] for idx in range(num_inputs)]

model_cls = ANFIS(n_inputs=num_inputs, 
                    n_mfs=[3]*num_inputs, 
                    mf_type='gaussmf',
                    input_ranges=input_ranges,
                    classification=True, 
                    learning_rate=5e-1,
                    lambda_l2=0.001, 
                    batch_size=128,
                    use_adaptive_lr=True)

# model_cls.fit(X_train_norm, 
#                 y_train, 
#                 epochs=500, 
#                 verbose=True,
#                 X_val=X_val_norm, 
#                 y_val=y_val, 
#                 early_stopping_patience=100)


#============================================================================================
# USE THIS TO FIT USING  the fit_metaheuristic method
#============================================================================================

# model_cls.fit_metaheuristic(X=X_train_norm,
#                             y=y_train,
#                             optimizer='pso',
#                             n_particles=100,
#                             n_iterations=500,
#                             X_val=X_val_norm,
#                             y_val=y_val)

In [ ]:
# Evaluate model on test set
y_pred = model_cls.predict(X_test_norm)
print(classification_report(y_test, y_pred, target_names=labels))
cm = confusion_matrix(y_test, y_pred)
ConfusionMatrixDisplay(cm, display_labels=labels).plot(cmap=plt.cm.Blues)

In [ ]:
# Visualize membership functions
fig = model_cls.plot_membership_functions()

In [ ]:
# Visualize training metrics
fig = model_cls.plot_metrics()

In [ ]:
# Visualize regularization effect
fig = model_cls.plot_regularization()

In [ ]:
# View learned fuzzy rules
df = model_cls.rules_to_dataframe()
df

In [ ]:
# Show fuzzy rules in table format
fig = model_cls.show_rules_table(figsize=(15,7.5))

In [ ]:
# Create grid for decision surface
x1_min, x1_max = X_train_norm[:, 0].min() - 0.5, X_train_norm[:, 0].max() + 0.5
x2_min, x2_max = X_train_norm[:, 1].min() - 0.5, X_train_norm[:, 1].max() + 0.5

xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max, 200),
                       np.linspace(x2_min, x2_max, 200))

# Make predictions on grid
print('Generating decision surface...')
Z = model_cls.predict(np.c_[xx1.ravel(), xx2.ravel()]).reshape(xx1.shape)
print('Surface generated!')

# Plot decision boundary
fig, ax = plt.subplots(figsize=(12, 10))

# Surface
contour = ax.contourf(xx1, xx2, Z, levels=[0, 0.5, 1],
                      colors=['#ffcccc', '#ccccff'], alpha=0.6)

# Boundary contour
ax.contour(xx1, xx2, Z, levels=[0.5], colors='black',
           linewidths=3, linestyles='solid')

# Points with error indication
y_test_pred = model_cls.predict(X_test_norm)
correct = y_test == y_test_pred
incorrect = ~correct

ax.scatter(X_test_norm[correct, 0], X_test_norm[correct, 1],
           c='green', marker='o', s=100, label='Correct',
           edgecolors='black', linewidth=1.5, alpha=0.8)
ax.scatter(X_test_norm[incorrect, 0], X_test_norm[incorrect, 1],
           c='orange', marker='X', s=150, label='Error',
           edgecolors='red', linewidth=2, alpha=0.9)

ax.set_xlabel('Petal Length (normalized)', fontsize=12)
ax.set_ylabel('Petal Width (normalized)', fontsize=12)
ax.set_title(f'ANFIS Decision Boundary\nErrors: {np.sum(incorrect)}/{len(y_test)}',
             fontsize=14, weight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()